### prototype_numba_tetra_project

Python-based prototype building off of <code>/notebooks/prototype_tetra_mapping</code>.
Will be using Numba for optimization of numerical routines.

#### Summary

The goal of this algorithm is to project voxels in 3D MR volume space to volumetric mesh space. To achieve this we use Monte Carlo sampling of tetrahedrons embedded in MR space to estimate the contributions of each voxel to a given tetrahedron. 

Uniform sampling of tetrahedrons is achieved using the parallelepiped folding algorithm [Generating Random Points in a Tetrahedron](http://vcg.isti.cnr.it/publications/papers/rndtetra_a.pdf). 

In [ ]:
import sys

In [ ]:
#If using jupyter hub
sys.path.insert(0,'/imaging/home/kimel/jjeyachandra/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/imaging/home/kimel/jjeyachandra/.conda/envs/3.6_tetra/lib/python3.6/site-packages/')

In [ ]:
import os
import gmsh
import numpy as np
import nibabel as nib
import nilearn as nil
from nilearn import plotting, image
from random import uniform
import matplotlib
from timeit import default_timer as timer
%matplotlib inline

In [ ]:
gmsh.initialize()

In [ ]:
f_tetra = '../data/simnibs_output/sub-CMH090.msh'
f_ribbon_r = '../data/sub-CMH090/ribbon/sub-CMH090_R_ribbon.nii.gz'
f_ribbon_l = '../data/sub-CMH090/ribbon/sub-CMH090_L_ribbon.nii.gz'
f_t1 = '../data/simnibs_output/m2m_sub-CMH090/T1fs_nu_conform.nii.gz'

### Pre-processing

In [ ]:
t1_img = image.load_img(f_t1)
affine = t1_img.affine

#Load in ribbon files and merge hemispheres
r_ribbon_img = image.load_img(f_ribbon_r)
l_ribbon_img = image.load_img(f_ribbon_l)
ribbon_img = image.math_img('a+b',a=r_ribbon_img,b=l_ribbon_img)
#plotting.view_img(ribbon_img,bg_img=t1_img)

In [ ]:
#Read gmsh MSH object
gmsh.open(f_tetra)

#Get tetrahedral volume 
tet_gm = (3,2)
tet_node_tag, tet_node_coord, tet_node_param = gmsh.model.mesh.getNodes(tet_gm[0],tet_gm[1])
tet_elem_tag, tet_elem_coord, tet_elem_param = gmsh.model.mesh.getElements(tet_gm[0],tet_gm[1])

# Get grey matter boundary surface
surf_gm = (2,2)
gm_node_tag, gm_node_coord, gm_node_param = gmsh.model.mesh.getNodes(surf_gm[0],surf_gm[1])
_, gm_elem_coord, gm_elem_param = gmsh.model.mesh.getElements(surf_gm[0],surf_gm[1])

# Get white matter boundary surface
surf_wm = (2,1)
wm_node_tag, wm_node_coord, wm_node_param = gmsh.model.mesh.getNodes(surf_wm[0], surf_wm[1])
_, wm_elem_coord, wm_elem_param = gmsh.model.mesh.getElements(surf_wm[0], surf_wm[1])

In [ ]:
#Pull ribbon data and coordinates
ribbon = ribbon_img.get_data()
x,y,z = np.where(ribbon != 0)

### Tetrahedral Projection

In [ ]:
from numba import jit, njit, vectorize, float64, int64, guvectorize, prange
from functools import partial

First re-structure the node --> coordinate mapping into numpy array types, numba doesn't support dictionary types

In [ ]:
#Why does parallel break?
@njit
def map_nodes(x, prop_array,out):
    '''
    Convenience function to remap a value according to a properties array
    Arguments:
        x                           Array
        prop_array                  A properties array in the desired order
        
    Description of prop_array
    
    prop_array is an (nx3) numpy array that stores the following information for the ith gmsh element
    [1] - minimum element node number
    [2] - maximum element node number
    [3] - number of element nodes
    
    This will remap the values such that it goes from  0 --> np.sum(prop_array[:,2]) 
    So that array-index based hashing can be used for fast coordinate mapping
    '''
    
    
    #Loop
    for i in np.arange(x.shape[0]):
        for j in np.arange(0,x.shape[1]):
            for k in np.arange(0,prop_array.shape[0]):
        
                if (x[i,j] >= prop_array[k,0]) & (x[i,j] <= prop_array[k,1]):
                    out[i,j] = x[i,j] - prop_array[k,0] + np.sum(prop_array[:k,2])

    return out

In [ ]:
tet_node_list = tet_elem_param[0].reshape((tet_elem_param[0].shape[0]//4, 4))

In [ ]:
#Step 1: Store min/max/len -- this will help us map values to node coordinates
min_tet = np.min(tet_node_tag)
max_tet = np.max(tet_node_tag)
len_tet = np.size(tet_node_tag)

min_gm = np.min(gm_node_tag)
max_gm = np.max(gm_node_tag)
len_gm = np.size(gm_node_tag)

min_wm = np.min(wm_node_tag)
max_wm = np.max(wm_node_tag)
len_wm = np.size(wm_node_tag)

#Property array to wrap up above features
prop_arr = np.array([
    [min_tet,max_tet,len_tet],
    [min_gm,max_gm,len_gm],
    [min_wm,max_wm,len_wm]
])

In [ ]:
#Set up inputs to projection algorithm
out = np.zeros_like(tet_node_list,dtype=np.int64)
shift_node_list = map_nodes(tet_node_list,prop_arr, out)
coord_arr = np.concatenate((tet_node_coord,gm_node_coord,wm_node_coord))
out_arr = np.zeros((shift_node_list.shape[0],np.unique(ribbon).shape[0]))

#### Main numba code

In [ ]:
@njit
def aabb_voxels(coords):
    '''
    Use axis-aligned boundary box in voxel space to identify candidate voxels
        coords                              (4,3) array containing tetrahedral coordinates in voxel space
    '''
    
    #Pre-allocate and store bounds
    min_vox = np.zeros((1,3),dtype=np.int)
    max_vox = np.zeros((1,3),dtype=np.int)
    min_vox[:] = np.floor(coords,axis=0).astype(np.int)
    max_vox[:] = np.ceil(coords,axis=0).astype(np.int)
    
    #Get voxel set
    x_range = np.arange(min_vox[0],max_vox[0]+1)
    y_range = np.arange(min_vox[1],max_vox[1]+1)
    z_range = np.arange(min_vox[2],max_vox[2]+1)
    vox_grid = np.meshgrid(x_range,y_range,z_range)
    
    #Format into colum vectors
    x_arr = vox_grid[0].reshape((1,vox_grid[0].size))
    y_arr = vox_grid[1].reshape((1,vox_grid[1].size))
    z_arr = vox_grid[2].reshape((1,vox_grid[2].size))
    
    #Pre-allocate and assign
    vox_arr = np.zeros((3,x_arr.shape[1]+y_arr.shape[1]+z_arr.shape[1]))
    vox_arr[0,:] = x_arr
    vox_arr[1,:] = y_arr
    vox_arr[2,:] = z_arr
    
    return vox_arr
    

In [ ]:
@njit
def homogenous_transform(coords,L):
    '''
    Transform into homogenous coordinates and apply linear map, will modify input!
        coords                              (1x3) array to transform
        L                                   Linear map to apply
    '''
    
    tmp = np.ones((4,coords.shape[0]),dtype=np.float32)
    tmp[:4,:] = coords.T
    tmp = np.matmul(L,tmp)
    coords[:,:] = tmp[:3,:]
    
    return coords

In [ ]:
@njit
def uniform_tet(coords):
    '''
    Argument:
        coords                A (4,3) matrix with rows representing nodes
    Output:
        point                 A random point inside the tetrahedral volume
    '''
    
    s = uniform(0,1)
    t = uniform(0,1)
    u = uniform(0,1)

    #First cut
    if (s+t > 1):
        s = 1.0 - s
        t = 1.0 - t
        
    #Second set of cuts  
    if (t+u > 1):
        tmp = u
        u = 1.0 - s - t
        t = 1.0 - tmp
    elif (s + t + u > 1):
        tmp = u 
        u = s + t + u - 1 
        s = 1 - t - tmp
        
    a = 1 - s - t - u

    return a*coords[0] + s*coords[1] + t*coords[2] + u*coords[3]

In [ ]:
@njit
def point_in_vox(point,midpoint,voxdim=1):
    '''
    Arguments:
        point                         Iterable of length 3
        midpoint                      Voxel midpoint
        voxdim                        Voxel dimensions, assuming isotropic
        
    Output:
        Boolean: True if point in voxel bounds
    '''
    
    halfvox = voxdim/2
    
    #Checks
    if (point[0] <= midpoint[0] - halfvox) or (point[0] >= midpoint[0] + halfvox):
        return False
    elif (point[1] <= midpoint[1] - halfvox) or (point[1] >= midpoint[1] + halfvox):
        return False
    elif (point[2] <= midpoint[2] - halfvox) or (point[2] >= midpoint[2] + halfvox):
        return False
    else:
        return True

In [ ]:
@njit
def estimate_partial_parcel(coord,vox,parcels,out_arr,n_iter=300):
    '''
    Arguments:
        coord               (4,3) indexable iterable of tetrahedral coordinates
        vox                 (n,3) indexable iterable of voxel coordinates
        parcels             (n,1) indexable iterable of parcel labels associated with jth voxel coordinate
        out_arr              Output array to be written into 
        iter                 Number of Monte-Carlo sampling interations
    Output:
        partial_parcel      (n,1) array containing voxel ownership of tetrahedron 

    For each tetrahedron we want to assign the value of the voxel 
    '''
    
    #Check degenerate case
    if np.unique(parcels).shape[0] == 1:
        out_arr[parcels[0]] = 1
        return out_arr
    
    #Shift tetrahedron to origin
    t = coord[0]
    coord = coord - t
    
    #Perform fixed monte carlo sampling
    for i in np.arange(0,n_iter):
        p = uniform_tet(coord)
        
        for j in np.arange(0,vox.shape[1]):
            if point_in_vox(p+t, vox[:,j]):
                out_arr[parcels[j]] += 1
                continue
                
    out_arr = out_arr/n_iter
    return out_arr

In [ ]:
@njit
def tetrahedral_projection(node_list,coord_arr,ribbon,affine,out_arr):
    '''
    Perform tetrahedral projection
        node_list                           List of tetrahedral nodes
        coord_arr                           Coordinate list (length=n) in groups of 3 for each node
        ribbon                              3D array containing parcels
        affine                              Affine transformation matrix associated with ribbon
        out_arr                             Output array
    '''
    
    #Compute inverse affine
    inv_affine = np.linalg.inv(affine)
    
    #Loop tetrahedrons
    t_coord = np.zeros((4,3),dtype=np.float32)
    for i in np.arange(0,node_list.shape[0]):
        
        #Get coordinates for nodes
        t_coord[0,:] = coord_arr[3*node_list[i,0]:(3*node_list[i,0])+3]
        t_coord[1,:] = coord_arr[3*node_list[i,1]:(3*node_list[i,1])+3]
        t_coord[2,:] = coord_arr[3*node_list[i,2]:(3*node_list[i,2])+3]
        t_coord[3,:] = coord_arr[3*node_list[i,3]:(3*node_list[i,3])+3]
        
        #Step 1: Transform coordinates (maybe use numpy)
        t_coord[0,:] = homogenous_transform(t_coord[0,:],inv_affine)
        t_coord[1,:] = homogenous_transform(t_coord[1,:],inv_affine)
        t_coord[2,:] = homogenous_transform(t_coord[2,:],inv_affine)
        t_coord[3,:] = homogenous_transform(t_coord[3,:],inv_affine)
        
        #Step 2: Perform axis-aligned boundary box finding
        vox_arr = aabb_voxels(coords)
        
        #Step 3: Get parcel values associated with voxels  then transform
        parcels = np.zeros((1,vox_arr.shape[1]))
        for j in np.arange(vox_arr.shape[1]):
            parcels[j] = ribbon[vox_arr[0,j],vox_arr[1,j],vox_arr[2,j]]
            vox_arr[:,j] = homogenous_transform(vox_arr[:,j],affine)
        
        #Step 4: Estimate partial parcels
        out_arr[i,:] = estimate_partial_parcel(coord,vox_arr,parcels,out_arr[i,:],300)
    
    return out_arr
        
        